# Data Science Workflow

## Set Up the Jupyter Notebook for Analysis

Note: We have our brand new package called swat - SAS Scripting Wrapper for Analytics Transfer - available on GitHub via pip install <br>
Copyright (c) 2017 SAS Institute Inc.

In [ ]:
import swat

In [ ]:
### SAS Viya CAS Server connection details

cashost='localhost'
casport=5570
casauth='~/.authinfo'

### Start CAS Session

s = swat.CAS(cashost, casport, authinfo=casauth)

In [ ]:
s

In [ ]:
# Load actionsets for analysis (for data prep, modeling, assessing)
actionsets = ['cardinality', 'sampling', 'fedSQL', 'decisionTree', 'regression', 'neuralNet', 'svm', 'astore','autotune']
actions = [s.loadactionset(i) for i in actionsets]

## Access Data - CAS

In [ ]:
# Define caslib
s.sessionprop.setsessopt(caslib='Public')

# data file
indata = 'HMEQ_NEW'

# CASTable view
castbl = s.CASTable(indata)

# Local data analysis

In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics  import confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Bring data locally
df = castbl.to_frame()

## Data Exploration and Preparation - Local

In [ ]:
hists = df.hist(figsize = (15, 10))

### set variable shortcuts

In [ ]:
# Define target variable
target = 'BAD'

# get categorical/numerical variables
input_vars = df.drop(target,axis=1).columns.values
kinds = np.array([dt.kind for dt in df.drop(target,axis=1).dtypes])

# Variable lists
categorical_vars = list(input_vars[kinds == 'O'])
numerical_vars = [i for i in list(input_vars[kinds != 'O']) if i not in ['Unique_ID','_PARTIND_']]
all_vars = categorical_vars + numerical_vars + [target]

### Prepare data for model

In [ ]:
# Split into training and validation / target and inputs
X_train = df[df['_PARTIND_'] == 1].drop(target, axis = 1)
X_valid = df[df['_PARTIND_'] == 0].drop(target, axis = 1)
y_train = df[df['_PARTIND_'] == 1][target]
y_valid = df[df['_PARTIND_'] == 0][target]

## Build model - Scikit-learn

In [ ]:
# Define data pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))])

categorical_transformer = Pipeline(steps=[
     ('imputer', SimpleImputer(strategy='most_frequent')),
     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numerical_vars),
    ('cat', categorical_transformer, categorical_vars)])

ml_pipe = Pipeline(steps=[
('preprocessor', preprocessor),
('model', GradientBoostingClassifier(random_state=3))])

fit = ml_pipe.fit(X_train, y_train)

### Assess model

In [ ]:
# put results in pandas dataframe
results_df = pd.DataFrame(dict(actual = y_valid, pred = ml_pipe.predict_proba(X_valid)[:,1]))

# Predict and assess model
gb_y_score           = ml_pipe.predict(X_valid)
gb_misclassification = 1 - accuracy_score(y_valid, gb_y_score)
gb_confusion_matrix  = confusion_matrix(y_valid, gb_y_score)

# Add Python model results to CAS to assess
pytbl = s.upload_frame(results_df, casout=dict(name='GBT_sklearn_predictions', replace=True))

In [ ]:
gb_misclassification

# Data Analysis on CAS

In [ ]:
castbl.head()

## Data Exploration and Preparation - CAS

In [ ]:
# Compute correlation matrix
corr = castbl.corr()

# Visualize with Seaborn
sns.set(style="white")

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

## Build models - CAS

In [ ]:
import collections

# Models to be performed
models = collections.OrderedDict()
models['dt']  = 'Decision Tree'
models['gbt'] = 'Gradient Boosting'
models['nn']  = 'Neural Network'
models['svm'] = 'Support Vector Machine'

# Impute missing values
castbl.dataPreprocess.impute(
    outVarsNamePrefix = 'IMP',
    methodContinuous  = 'MEDIAN',
    methodNominal     = 'MODE',
    inputs            = categorical_vars + numerical_vars,
    copyAllVars       = True,
    casOut            = dict(name = 'hmeq_imputed', replace = True)
)
castbl_imp = s.CASTable('hmeq_imputed')

In [ ]:
# Set key-word argument shortcuts (common model inputs)
## For models that can handle missing values (decision tree, gradient boosting)
params = dict(
    table    = dict(name = indata, where = '_PARTIND_ = 1'), 
    target   = target, 
    inputs   = categorical_vars + numerical_vars, 
    nominals = categorical_vars,
)

## For models that can't have missing values (neural network, support vector machine)
casinfo = castbl_imp.columninfo()['ColumnInfo']
imp_params = dict(
    table    = dict(name = 'hmeq_imputed', where = '_PARTIND_ = 1'), 
    target   = target, 
    inputs   = [x for x in list(casinfo['Column']) if x != target and ('IMP_' in x)], 
    nominals = [x for x in list(casinfo[casinfo.Type == 'varchar']['Column']) if x != target and ('IMP' in x)],
)

### Decision Tree - CAS

In [ ]:
dt = s.decisionTree.dtreeTrain(**params, varImp = True, casOut = dict(name = 'dt_model', replace = True))

### Gradient Boosting - CAS

In [ ]:
#s.dropTable('gradboost_astore')
gbt = s.decisionTree.gbtreeTrain(**params, seed = 5, casOut = dict(name = 'gbt_model', replace = True), savestate = 'gradboost_astore')

### Neural Network - CAS

In [ ]:
nn = s.neuralNet.annTrain(**imp_params, seed = 1, casOut = dict(name = 'nn_model', replace = True))

### Support Vector Machine - CAS

In [ ]:
svm = s.svm.svmTrain(**imp_params, seed = 1, kernel = 'polynomial', printtarget=True, id = ['_PARTIND_', 'Unique_ID'], savestate = dict(name = 'svm_model', replace = True))

# Model Comparison

## Score the Models on Validation Data - CAS/Python

In [ ]:
def score_model(model):
    score = dict(
        table      = 'hmeq_imputed',
        modelTable = model + '_model',
        copyVars   = [target, '_PARTIND_'],
        casOut     = dict(name = '_scored_' + model, replace = True),
        encodeName = True
    )
    return score

### Decision Tree
s.decisionTree.dtreeScore(**score_model('dt'))
### Gradient Boosting
s.decisionTree.gbtreeScore(**score_model('gbt'))
### Neural Network
s.neuralNet.annScore(**score_model('nn'))
### Support Vector Machine
svmscore = castbl_imp.astore.score(rstore = 'svm_model', out = dict(name = '_scored_svm_1', replace = True))

In [ ]:
joined = s.CASTable('_scored_svm_1').merge(castbl_imp[['Unique_ID','BAD']],on='Unique_ID')
s.dropTable('_scored_svm')
s.table.altertable(name = joined.name, rename='_scored_svm')

## Assess the Performance - CAS/Python

In [ ]:
# Model assessment function
def assess_model(model):
    assess = s.percentile.assess(
        table    = dict(name = '_scored_' + model, where = '_PARTIND_ = 0'),
        inputs   = 'p_' + target,      
        response = target,
        event    = '1',
    )
    return assess

# Loop through the models and append to the roc_df dataframe
roc_df  = pd.DataFrame()
for i in range(len(models)):
    tmp = assess_model(list(models)[i])
    tmp.ROCInfo['Model'] = list(models.values())[i]
    roc_df = pd.concat([roc_df, tmp.ROCInfo])

## Compare Misclassification & ROC Curves - CAS/Python

In [ ]:
# Assess the Python model using CAS
python_assess = pytbl.percentile.assess(
    inputs   = 'pred',      
    response = 'actual',
    event    = '1',   
)
python_assess.ROCInfo['Model'] = 'Gradient Boosting - sklearn'
roc_df['Model'] = roc_df['Model'] + ' - CAS'
roc_df = pd.concat([roc_df, python_assess.ROCInfo])
roc_df['Misclassification'] = 1 - roc_df['ACC']

print('\n', 'Misclassification Rate Comparison'.center(37, ' '))
miss = roc_df[round(roc_df['CutOff'], 2) == 0.5][['Model', 'Misclassification']].reset_index(drop = True)
miss.sort_values('Misclassification')

In [ ]:
# Plot ROC curve
plt.figure(figsize = (7, 6))
for key, grp in roc_df.groupby(['Model']):
    plt.plot(grp['FPR'], grp['Sensitivity'], label = key + ' (C = %0.2f)' % grp['C'].mean())
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('False Postivie Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.title('ROC Curve (using validation data)');

## Automated Machine Learning

## dsAutoML - CAS

In [ ]:
s.loadactionset('dataSciencePilot')
trans_out = dict(name = 'trans_out', replace=True)
feat_out = dict(name = 'feat_out', replace=True)
pipeline_out = dict(name = 'pipeline_out', replace=True)
#save_state_out = dict(name = 'save_state_out', replace=True)
automl_model = dict(name = 'automl_model', replace=True) 

In [ ]:
s.dataSciencePilot.dsAutoMl(table=castbl, target=target,
                            modelTypes=["DECISIONTREE", "FOREST", "GRADBOOST", "NEURALNET"], 
                            transformationOut = trans_out,
                            featureOut = feat_out,
                            pipelineOut = pipeline_out,
                            saveState = automl_model)

In [ ]:
s.fetch(table='pipeline_out', to=3)

# Register Model

## Add model to repository - CAS

In [ ]:
from sasctl import Session
from sasctl.tasks import register_model
from sasctl.services import model_repository

In [ ]:
astore = s.CASTable('gradboost_astore')
with Session('http://sasserver.demo.sas.com', 'sasdemo', 'Orion123'):
    register_model(astore, 'GBT Model swat', '4. Manage churn models')

In [ ]:
model = ml_pipe
with Session('http://sasserver.demo.sas.com', 'sasdemo', password):
    register_model(model, 'GBT Model sklearn', '4. Manage churn models')